# PyTorch Benckmark Prediction
Linear Regression Model
---

Credits:
- https://towardsdatascience.com/linear-regression-with-pytorch-eb6dedead817


In [1]:
#Torch
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch import Tensor
import torch.nn as nn
import torch.optim as optim

# sklearn
from sklearn.model_selection import train_test_split

#Scipy
import scipy

#Numpy
import numpy as np

#Pandas
import pandas as pd

#Matplot
import matplotlib.pyplot as plt

#Librosa
import librosa
import librosa.display

# Other  
import os

# Local
from source_pytorch.LinearDataset import LinearDatset
from source_pytorch.LinearModel import LinearModel

In [2]:
#Set the parameters and load the data
setting = {
  "audio_duration"  :   4,
  "n_mfcc"          :   40,
  "sampling_rate"   :   44100,
  "audio_duration"  :   4,
  "number_samplig"  :   354,
  "dataroot"        :   ".",
  "batch_size"      :   10,
  "epochs"          :   50
}

In [3]:
#Get the reference
ref = pd.read_csv("./ReferenceData.csv")#[:37]
#Print head / test successfull load
ref.head()

,label,fname
0,fearful,03-01-06-01-02-02-02.wav
1,angry,03-01-05-01-02-01-16.wav
2,surprise,03-01-08-01-01-01-14.wav
3,fearful,03-01-06-01-02-02-16.wav
4,angry,03-01-05-01-02-01-02.wav


In [4]:
#Split the Train and Test dataset
X_train, X_test, y_train, y_test = train_test_split(ref["fname"]
                                                    ,ref["label"]
                                                    , test_size=0.25
                                                    , shuffle=True
                                                    , random_state=50
                                                   )
#Split the Train and Validation dataset
X_train, X_valid , y_train, y_valid = train_test_split(X_train
                                                    , y_train
                                                    , test_size=0.25
                                                    , shuffle=True
                                                    , random_state=50
                                                   )


#Merge data and prediction in 1 dataset for train validation and test
train_df = pd.concat([X_train,y_train], axis = 1)  
valid_df =  pd.concat([X_valid,y_valid], axis = 1)
test_df = pd.concat([X_test,y_test], axis = 1)

print('Dataset Len:\t Train - {} \tValid Dataset - {} \tTest Dataset - {}'.format(len(train_df), len(valid_df), len(test_df)))

Dataset Len:	 Train - 810 	Valid Dataset - 270 	Test Dataset - 360


In [5]:
#Compose DataLoader
train_dataloader = DataLoader(
    LinearDatset(
        dataframe=train_df,
        n_melspec = setting["n_mfcc"], 
        sampling_rate = setting["sampling_rate"], 
        audio_duration= setting["audio_duration"], 
        number_samples= setting["number_samplig"]
        ), 
    batch_size= setting["batch_size"], 
    shuffle=True, 
    num_workers=0)

valid_dataloader = DataLoader(
    LinearDatset(
        dataframe=valid_df,
        n_melspec = setting["n_mfcc"], 
        sampling_rate = setting["sampling_rate"], 
        audio_duration= setting["audio_duration"], 
        number_samples= setting["number_samplig"]
    ), 
    batch_size=setting["batch_size"], 
    shuffle=True, 
    num_workers=0
    )

In [6]:
#Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LinearModel()
model.to(device)

print(model)

LinearModel(
  (linear): Linear(in_features=13800, out_features=8, bias=True)
)


In [7]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [8]:
%%time
#Credits: https://github.com/udacity/deep-learning-v2-pytorch/blob/master/convolutional-neural-networks/cifar-cnn/cifar10_cnn_solution.ipynb

valid_loss_min = np.Inf # track change in validation loss


for epoch in range(1, setting["epochs"]+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0

    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_dataloader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)

    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_dataloader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)

    # calculate average losses
    train_loss = train_loss/len(train_dataloader.sampler)
    valid_loss = valid_loss/len(valid_dataloader.sampler)
    
    # print training/validation statistics 
    print('\n Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss


 Epoch: 1 	Training Loss: 5069.969673 	Validation Loss: 5907.600188
Validation loss decreased (inf --> 5907.600188).  Saving model ...

 Epoch: 2 	Training Loss: 3761.992362 	Validation Loss: 2157.638645
Validation loss decreased (5907.600188 --> 2157.638645).  Saving model ...

 Epoch: 3 	Training Loss: 2540.186466 	Validation Loss: 3884.556302

 Epoch: 4 	Training Loss: 1969.485092 	Validation Loss: 2658.230613

 Epoch: 5 	Training Loss: 2342.914899 	Validation Loss: 5483.300302

 Epoch: 6 	Training Loss: 3168.877873 	Validation Loss: 4568.887533

 Epoch: 7 	Training Loss: 2433.802193 	Validation Loss: 3487.747649

 Epoch: 8 	Training Loss: 2810.761915 	Validation Loss: 3656.353674

 Epoch: 9 	Training Loss: 1818.549708 	Validation Loss: 2972.077361

 Epoch: 10 	Training Loss: 2220.330833 	Validation Loss: 3063.745949

 Epoch: 11 	Training Loss: 2369.596281 	Validation Loss: 3391.804091

 Epoch: 12 	Training Loss: 2521.363591 	Validation Loss: 6348.427915

 Epoch: 13 	Training Loss:

In [9]:
model.load_state_dict(torch.load('model_cifar.pt'))

<All keys matched successfully>

In [10]:
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

# Load Test Data
test_dataloader = DataLoader(
    LinearDatset(
        dataframe=test_df,
        n_melspec = setting["n_mfcc"], 
        sampling_rate = setting["sampling_rate"], 
        audio_duration= setting["audio_duration"], 
        number_samples= setting["number_samplig"]
        ), 
    batch_size= setting["batch_size"], 
    shuffle=True, 
    num_workers=0)



In [11]:
# Get the model and evaluate
model.eval()

right_pred = 0
total_pred = 0

for data, target in test_dataloader:
    output = model(data)
    loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())

    for i in range(setting["batch_size"]):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

    total_pred += target.size(0)
    right_pred += (pred == target).sum().item()
        
# average test loss
test_loss = test_loss/len(test_dataloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

classes = ['angry', 'surprise', 'fearful', 'neutral', 'calm', 'happy', 'sad', 'disgust']

for i in range(len(classes)):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('Test Total Accuracy %2d%%' % (100 * right_pred / total_pred))

Test Loss: 2346.813819

Test Accuracy of angry: 32% (14/43)
Test Accuracy of surprise:  0% ( 0/45)
Test Accuracy of fearful:  0% ( 0/47)
Test Accuracy of neutral:  0% ( 0/51)
Test Accuracy of  calm:  0% ( 0/54)
Test Accuracy of happy: 37% (15/40)
Test Accuracy of   sad:  0% ( 0/33)
Test Accuracy of disgust:  0% ( 0/47)
Test Total Accuracy  8%
